# SQL Alchemy Core

## Table of Contents

  1. [Introduction](#Introduction)
  2. [Coding](#Coding)
    1. [Connecting](#Connecting)
    2. [Define and create tables](#Define-and-Create-Tables)
    3. [Insert Expressions](#Insert-Expressions)
    4. [Executing the Query](#Executing-the-Query)
    5. [Executing Multiple Statements](#Executing-Multiple-Statements)
    6. [Select Query](#Select-Query)
    7. [Selecting Specific Columns](#Selecting-Specific-Columns)
    8. [Operators](#Operators)
    9. [Using Conjunctions](#Using-Conjunctions)
    10. [Using Joins](#Using-Joins)

## Introduction

This is a library that gives Python developers the full power and flexibility of SQL. It provides us with a full
suite of well known enterise-level persisentence patterns adapted into simple Python domain langauge.

With SQlAlcemy the SQL databases will behave less like object collections the more size and performance matter,
Object collections behave less like tables and rows the more abstractions matter. SQLAlchemy aims to accomodate
both of these principles.

SQLAlchemy considers the databases to be a relational algebra engine, not just a collection of tables. Rows can
not only be selcted from tables but also joins and other select statements. Any of these units can be composed
into a larger structure. The language SQLAlchemy uses builds on this concept from it's core.

SQLAlchemy is most famous for its object-relational mapper (ORM). an optional component that provides the **data mapper pattern**
, where classes can be mapped to the database in open-ended, multiple ways - allowing the object model and database
schema to develop in a cleanly decoupled way from the beginning.

SQLAlchemy overall approach to these platforms is entirely different from that of most other SQL tools, rooted in
a so-called **complimentary**- orientated approach. All processes are **fully-exposed** within a series of composable
, transparent tools.

## Coding

### Connecting

For this example I will be using an in Memory database like SQLLite.

We will start with importing the library and connecting to the database in

In [68]:
from sqlalchemy import create_engine
engine = create_engine("sqlite:///data/Example.db", echo=True)

The echo statement setups up the SQLAlchemy logging which will allow us to see all
of the SQL statements generated.

The return of the **create_engine** method is an instance of an engine and represents
the core interfact to the database.

It is important to know that the creation of the engine has not actually attempted to
connect to the database just yet and will only connect once an execution is requested.

We can now use either `engine.connect()`or `engine.execute()`  to connect to the database.

### Define and Create Tables

The SQL Expression Language constructs its expressions in most cases against table columns. In
SQL Alchemy, a column is most often represented by an object called `Column` and in all cases
a `Column` is associated with a `Table`.  A collection of `Table` objects and their associated
child objects is referred to as **database metadata**.

While I will be showing how we can lay out the tables in this section, SQLAlchemy can also import
whole sets of table objects automatically from an existing database (This process is called **table reflection**).

We define all of our tables within a catalog called `MetaData`, using the `Table` construct. For this
example we will be creating a user table as well as a place to store their email addresses.

In [69]:
from sqlalchemy import Table, Column, Integer, String, MetaData, ForeignKey

metadata= MetaData()
users = Table(
    "users",
    metadata,
    Column(
        "id",
        Integer,
        primary_key=True
           ),
    Column(
        "name",
        String
    ),
    Column(
        "fullname",
        String
    )
              )
addresses = Table(
    "Addresses",
    metadata,
    Column(
        "id",
        Integer,
        primary_key=True
    ),
    Column(
        "user_id",
        None,
        ForeignKey("users.id")
    ),
    Column(
        "email_addresses",
        String,
        nullable=False
    )
)

The next step is to tell the `MetaData` that we would like to create our selection of tables
for real inside the SQLLite database. We will use the `create_all()` passing through the `engine`
instance which points to the database. This will also check for the presence of all of the tables
first before creating, so it is safe to be called multiple times.

In [70]:
metadata.create_all(engine)

2021-06-01 21:35:21,082 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2021-06-01 21:35:21,083 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("users")
2021-06-01 21:35:21,083 INFO sqlalchemy.engine.Engine [raw sql] ()
2021-06-01 21:35:21,085 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("Addresses")
2021-06-01 21:35:21,085 INFO sqlalchemy.engine.Engine [raw sql] ()
2021-06-01 21:35:21,087 INFO sqlalchemy.engine.Engine COMMIT


With SQLlite and PostgreSql a length is not required for the creation of Strings but are required
for the other database formats. We therefore need to adjust the specification of the Strings in
our `Columns` to include this detail by using `String(50)` for instance to create a **varchar**
of size 50.

### Insert Expressions

The First SQL expression we will create is the insert construct, which represents an insert statement.

we can do this using the tables we have created in the previous section.

In [71]:
ins = users.insert()

To be able to see the SQL that this expression creates we can use the following code:

In [72]:
str(ins)

'INSERT INTO users (id, name, fullname) VALUES (:id, :name, :fullname)'

While we can see a nicely generated SQL query has been inserted for the insert, there is one issue with the
current query - the generated statement is also trying to insert into the id. We can sort this out by
specifying the values that need to be inserted. The best way to do this can be seen below:

In [73]:
ins = users.insert().values(name="Adam", fullname="Adam Borlase")
str(ins)


'INSERT INTO users (name, fullname) VALUES (:name, :fullname)'

Now we can see that the columns we are inserting into exclude the id Column.

While we specified the data that needed to be inserted into the Columns it was not rendered into the populated
SQL string. Instead we got named binded parameters. As it turns out the data is stored instide our `Insert` construct
and will only come out when it is executed.

Since the data consists of literal values, SQLAlchemy automatically generates bind parameters for them and we can
peek at the data for now by looking at the compiled form of the statement.

In [74]:
ins.compile().params


{'name': 'Adam', 'fullname': 'Adam Borlase'}

### Executing the Query

the interesting part of the `Insert` query is the execution of the statement. This is performed using a
database connection which is represented by the `Connection` object. To get the connection object we will use the
`Engine`.

In [75]:
conn = engine.connect()
conn

The `Connection` object represents an actively checked out DBAPI connection resource. We will now feed it
our insert object and see what happens:

In [76]:
result = conn.execute(ins)

2021-06-01 21:35:21,124 INFO sqlalchemy.engine.Engine INSERT INTO users (name, fullname) VALUES (?, ?)
2021-06-01 21:35:21,125 INFO sqlalchemy.engine.Engine [generated in 0.00067s] ('Adam', 'Adam Borlase')
2021-06-01 21:35:21,126 INFO sqlalchemy.engine.Engine COMMIT


Now the insertion has been issued to the database. We specified a `result` variable when we executed the query,
this result is known as a `CursorResult` object and is analogous to the DBAPI cursor object. In the case of an
insert we get important information such as the primary key which were generated from our statement using
`CursorResult.inserted_primary_key

We can see the results below:

In [77]:
result.inserted_primary_key

(9,)

The value of 1 is automatically generated as we did not specify the value to be inserted into the primary key of the id
column in our insert statement.

### Executing Multiple Statements

The original `Insert` example was intentionally drawn out to show some various behaviours of expression language
constructs. In the usual case an `Insert` statement is usually compiled against the parameters sent to the `execute()`
method on `Connection`, so there is no need to use the `values` keyword with the `insert`. We will now create an `insert`
statement and use it in the normal way.

In [78]:
ins =  users.insert()
result = conn.execute(
    ins,
    {"name":"Dominique",
     "fullname":"Dominique Freeborough"}
)

2021-06-01 21:35:21,136 INFO sqlalchemy.engine.Engine INSERT INTO users (name, fullname) VALUES (?, ?)
2021-06-01 21:35:21,137 INFO sqlalchemy.engine.Engine [generated in 0.00070s] ('Dominique', 'Dominique Freeborough')
2021-06-01 21:35:21,138 INFO sqlalchemy.engine.Engine COMMIT


Again we can return a `CursorResult` object in order to access some of the details required for further processes later.

In [79]:
result.inserted_primary_key

(10,)

To issue many inserts using the DBAPI we can use the `executemany()` method, which allows us to send a list of dictionaries each containing a
distinct set of parameters to be inserted into, We can now do this with the email addresses:

In [80]:
conn.execute(
    addresses.insert(),
    {"user_id": 1, "email_addresses": "scod101@gmail.com"},
    {"user_id": 1, "email_addresses": "a.c.borlase@gmail.com"},
    {"user_id": 2, "email_addresses": "Name.Surname@gmail.com"}
)

2021-06-01 21:35:21,147 INFO sqlalchemy.engine.Engine INSERT INTO "Addresses" (user_id, email_addresses) VALUES (?, ?)
2021-06-01 21:35:21,148 INFO sqlalchemy.engine.Engine [generated in 0.00075s] ((1, 'scod101@gmail.com'), (1, 'a.c.borlase@gmail.com'), (2, 'Name.Surname@gmail.com'))
2021-06-01 21:35:21,149 INFO sqlalchemy.engine.Engine COMMIT


When we do as execution like above the key element is to ensure the dictionaries have the same number
of keys in the execution. This is because the insertion is compiled against the first dictionary in the `Insert`.

The `Executemany()` construct is avaliable for each of the `insert()`, `update()` and `delete()` constructs.

### Select Query

We began with inserts just so the test database had some data in it. The more interesting part of the data is the selection of it.
The main constuct used when doing a select is the `select()` function.

In [81]:
from sqlalchemy.sql import select
s = select(users)
result = conn.execute(s)

2021-06-01 21:35:21,155 INFO sqlalchemy.engine.Engine SELECT users.id, users.name, users.fullname 
FROM users
2021-06-01 21:35:21,156 INFO sqlalchemy.engine.Engine [generated in 0.00062s] ()


In the code above we execute a basic select statement call placing the users table within the columns clause of the select
method and then executing. SQLAlchemy generates a nice SQL query for us with selecting all of the columns as well as a from
statement.

Again the result is a `CursorResult` object which acts much like a DBAPI cursor including methods such as `fetchone()` and
`fetchall()`

In [82]:
for row in result:
    print(row)

(1, 'Adam', 'Adam Borlase')
(2, 'Dominique', 'Dominique Freeborough')
(3, 'Adam', 'Adam Borlase')
(4, 'Dominique', 'Dominique Freeborough')
(5, 'Adam', 'Adam Borlase')
(6, 'Dominique', 'Dominique Freeborough')
(7, 'Adam', 'Adam Borlase')
(8, 'Dominique', 'Dominique Freeborough')
(9, 'Adam', 'Adam Borlase')
(10, 'Dominique', 'Dominique Freeborough')


We can see from the above that each of the rows is returned as a simple tuple-like result and the best way to
access the individual values is by using a tuple assignment.

In [83]:
result = conn.execute(s)
for id, name, fullname in result:
    print(f"Name: {name}, Full Name: {fullname}")

2021-06-01 21:35:21,164 INFO sqlalchemy.engine.Engine SELECT users.id, users.name, users.fullname 
FROM users
2021-06-01 21:35:21,165 INFO sqlalchemy.engine.Engine [cached since 0.009781s ago] ()
Name: Adam, Full Name: Adam Borlase
Name: Dominique, Full Name: Dominique Freeborough
Name: Adam, Full Name: Adam Borlase
Name: Dominique, Full Name: Dominique Freeborough
Name: Adam, Full Name: Adam Borlase
Name: Dominique, Full Name: Dominique Freeborough
Name: Adam, Full Name: Adam Borlase
Name: Dominique, Full Name: Dominique Freeborough
Name: Adam, Full Name: Adam Borlase
Name: Dominique, Full Name: Dominique Freeborough


To access the columns via a name strings, we can either `Row._mapping` view may be used that provides dictionary
like access.

In [84]:
result = conn.execute(s)
row = result.fetchone()
print("Name: ", row._mapping['name'], " Full Name: ", row._mapping["fullname"])

2021-06-01 21:35:21,170 INFO sqlalchemy.engine.Engine SELECT users.id, users.name, users.fullname 
FROM users
2021-06-01 21:35:21,171 INFO sqlalchemy.engine.Engine [cached since 0.01564s ago] ()
Name:  Adam  Full Name:  Adam Borlase


As the `Row` is a tuple object we can also use integer slices to access the information:

In [85]:
print("Name: ", row[1], " Full name: ", row[2])

Name:  Adam  Full name:  Adam Borlase


A more specialised method to access the individual columns is to use the individual mapping key that we used when creating the table.

An example can be seen below:


In [86]:
for row in conn.execute(s):
    print("Name: ", row._mapping[users.c.name], " Full Name: ", row._mapping[users.c.fullname])


2021-06-01 21:35:21,179 INFO sqlalchemy.engine.Engine SELECT users.id, users.name, users.fullname 
FROM users
2021-06-01 21:35:21,179 INFO sqlalchemy.engine.Engine [cached since 0.02423s ago] ()
Name:  Adam  Full Name:  Adam Borlase
Name:  Dominique  Full Name:  Dominique Freeborough
Name:  Adam  Full Name:  Adam Borlase
Name:  Dominique  Full Name:  Dominique Freeborough
Name:  Adam  Full Name:  Adam Borlase
Name:  Dominique  Full Name:  Dominique Freeborough
Name:  Adam  Full Name:  Adam Borlase
Name:  Dominique  Full Name:  Dominique Freeborough
Name:  Adam  Full Name:  Adam Borlase
Name:  Dominique  Full Name:  Dominique Freeborough


The `CursorResult` object features an auto-close behaviour that closes the underlying DBAPI `cursor` object when all of the pending result
rows have been fetched. if we ever need to close the object prior to fetching all of the result objects we can use `CursorResult.close()`

In [87]:
result.close()

### Selecting Specific Columns

If we would like to have more carefully control of the columns that are included in the select query that are generated
from SQLAlchemy we can reference the individual `Column` object from the `Table`. These are avaliable as attributes off
the `c` attribute off of the `Table` object.

In [88]:
s = select(users.c.name, users.c.fullname)
results = conn.execute(s)
for row in results:
    print(row)

2021-06-01 21:35:21,189 INFO sqlalchemy.engine.Engine SELECT users.name, users.fullname 
FROM users
2021-06-01 21:35:21,190 INFO sqlalchemy.engine.Engine [generated in 0.00063s] ()
('Adam', 'Adam Borlase')
('Dominique', 'Dominique Freeborough')
('Adam', 'Adam Borlase')
('Dominique', 'Dominique Freeborough')
('Adam', 'Adam Borlase')
('Dominique', 'Dominique Freeborough')
('Adam', 'Adam Borlase')
('Dominique', 'Dominique Freeborough')
('Adam', 'Adam Borlase')
('Dominique', 'Dominique Freeborough')


As we can see from the generated SQL statement it knows that we are only interested in the information from the
users table and hence only requests this table. Let's try to select both of our tables and see what happens:

In [89]:
s = select(users,addresses)
results = conn.execute(s)
for row in results:
    print(row)

2021-06-01 21:35:21,195 INFO sqlalchemy.engine.Engine SELECT users.id, users.name, users.fullname, "Addresses".id AS id_1, "Addresses".user_id, "Addresses".email_addresses 
FROM users, "Addresses"
2021-06-01 21:35:21,196 INFO sqlalchemy.engine.Engine [generated in 0.00056s] ()
(1, 'Adam', 'Adam Borlase', 1, 1, 'scod101@gmail.com')
(1, 'Adam', 'Adam Borlase', 2, 1, 'a.c.borlase@gmail.com')
(1, 'Adam', 'Adam Borlase', 3, 2, 'Name.Surname@gmail.com')
(1, 'Adam', 'Adam Borlase', 4, 1, 'scod101@gmail.com')
(1, 'Adam', 'Adam Borlase', 5, 1, 'a.c.borlase@gmail.com')
(1, 'Adam', 'Adam Borlase', 6, 2, 'Name.Surname@gmail.com')
(1, 'Adam', 'Adam Borlase', 7, 1, 'scod101@gmail.com')
(1, 'Adam', 'Adam Borlase', 8, 1, 'a.c.borlase@gmail.com')
(1, 'Adam', 'Adam Borlase', 9, 2, 'Name.Surname@gmail.com')
(1, 'Adam', 'Adam Borlase', 10, 1, 'scod101@gmail.com')
(1, 'Adam', 'Adam Borlase', 11, 1, 'a.c.borlase@gmail.com')
(1, 'Adam', 'Adam Borlase', 12, 2, 'Name.Surname@gmail.com')
(1, 'Adam', 'Adam Borla

/Users/adamborlase/opt/anaconda3/envs/surveyor/lib/python3.7/site-packages/ipykernel_launcher.py:2: SAWarning: SELECT statement has a cartesian product between FROM element(s) "users" and FROM element "Addresses".  Apply join condition(s) between each element to resolve.
  


While it has put both of the tables in the select query it has made a mess of the results as there is no
join on the tables so the concatertation has happened with putting all of the data next to each other.

In order to sort this out we should use a join (Which will be explaned later) for the mean time lets explore how to
use a where statement. This can be done using `select().where()`

In [90]:
s = select(users, addresses).where(users.c.id == addresses.c.user_id)
results = conn.execute(s)
for row in results:
    print(row)

2021-06-01 21:35:21,210 INFO sqlalchemy.engine.Engine SELECT users.id, users.name, users.fullname, "Addresses".id AS id_1, "Addresses".user_id, "Addresses".email_addresses 
FROM users, "Addresses" 
WHERE users.id = "Addresses".user_id
2021-06-01 21:35:21,210 INFO sqlalchemy.engine.Engine [generated in 0.00065s] ()
(1, 'Adam', 'Adam Borlase', 1, 1, 'scod101@gmail.com')
(1, 'Adam', 'Adam Borlase', 2, 1, 'a.c.borlase@gmail.com')
(2, 'Dominique', 'Dominique Freeborough', 3, 2, 'Name.Surname@gmail.com')
(1, 'Adam', 'Adam Borlase', 4, 1, 'scod101@gmail.com')
(1, 'Adam', 'Adam Borlase', 5, 1, 'a.c.borlase@gmail.com')
(2, 'Dominique', 'Dominique Freeborough', 6, 2, 'Name.Surname@gmail.com')
(1, 'Adam', 'Adam Borlase', 7, 1, 'scod101@gmail.com')
(1, 'Adam', 'Adam Borlase', 8, 1, 'a.c.borlase@gmail.com')
(2, 'Dominique', 'Dominique Freeborough', 9, 2, 'Name.Surname@gmail.com')
(1, 'Adam', 'Adam Borlase', 10, 1, 'scod101@gmail.com')
(1, 'Adam', 'Adam Borlase', 11, 1, 'a.c.borlase@gmail.com')
(2, 

This is looking a lot better for the original results that we had.

If we look at the created SQL query we now have a where statement at the end of the select which evaluates
if the id of user is equal to user_id in the addresses table. If we look at the expression we can see that
it is using a python equality operator between the two columns. If we check what SQLAlchemy is doing with
the equality we will see it is not creating a true or false but rather an object:

In [91]:
users.c.id == addresses.c.user_id

In [92]:
str(users.c.id == addresses.c.user_id)

'users.id = "Addresses".user_id'

From this we can see that everything we work with creates the same type of object based on the base class of the
`ColumnElement`.

### Operators

Since we have already seen how the equality in SQLAlchemy works we can now go through it's capabilities.

Previously we created an equality based on the columns now lets see what happens if we have a literal value:

In [93]:
print(users.c.id == 1)

users.id = :id_1


The result is know as a bind parameter. we can use a similar trick to the insert section to see the results:

In [94]:
(users.c.id == 1).compile().params

{'id_1': 1}

With SQLAlchemy we can use a lot more of the python operators in the creation of the statements and not just the equals operator.

for instance we also have the not equal to and greater than:

In [95]:
print(users.c.id !=7)
print(users.c.name == None)
print("fred" > users.c.name)

users.id != :id_1
users.name IS NULL
users.name < :name_1


Just note that the None value is converted to is null when using it in the SQLAlchemy.

If we use an addition operator we will get an addition expression:

In [96]:
print(users.c.id + users.c.id)

users.id + users.id


The next key element to note is the type of column also matters, for instance if we use an addition on two string columns we will get
a different result than when we did the same thing to the Integer column.

In [97]:
print(users.c.name + users.c.fullname)

users.name || users.fullname


The `||` in the created expression stands for the concaternation operator used on most databases.

The produced expression is applicable to a mysql database versus the original result which was based on our SQLlite database.

if we ever need to use an operator that looks for a String match we can use the like operator:

In [98]:
print(users.c.name.like("%ed"))

users.name LIKE :name_1


While there are many other operators we can use, we can now see this in action.

In [99]:
s = select(users).where(users.c.id==1)
results = conn.execute(s)
for row in results:
    print(row)

2021-06-01 21:35:21,259 INFO sqlalchemy.engine.Engine SELECT users.id, users.name, users.fullname 
FROM users 
WHERE users.id = ?
2021-06-01 21:35:21,259 INFO sqlalchemy.engine.Engine [generated in 0.00060s] (1,)
(1, 'Adam', 'Adam Borlase')


### Using Conjunctions

Conjunctions are mostly known to be commonly used words such as `AND` and `OR` that put multiple things together a little
more. We can now start seeing the usage of these conjunctions:

In [100]:
from sqlalchemy import and_, or_, not_
print(and_(
    users.c.name.like("%j"),
    users.c.id == addresses.c.user_id,
    or_(
        addresses.c.email_addresses == "wendy@aol.com",
        addresses.c.email_addresses == "jacky@yahoo.com"
    ),
    not_(users.c.id>5)
    )
)

users.name LIKE :name_1 AND users.id = "Addresses".user_id AND ("Addresses".email_addresses = :email_addresses_1 OR "Addresses".email_addresses = :email_addresses_2) AND users.id <= :id_1


### Using Joins

The next big cornerstone to a select Query is a join statement. We have already done this with a where statement,
but sometimes it may be more useful to include a join attribute in the SQLAlchemy coding to assist us. This can be done
`FromClause.join()` or `FromClause.outerjoin()`.

A simple example is the following:

In [101]:
str(users.join(addresses))

'users JOIN "Addresses" ON users.id = "Addresses".user_id'

Since we specified a foreign key in the table creation we can see that SQLAlchemy has already filled in the joining columns
for us in the join statement.

If we ever need to specify the columns in the join we can modify the syntax a bit like the following:

In [102]:
str(users.join(
    addresses,
    addresses.c.email_addresses.like(users.c.name + '%')
))

'users JOIN "Addresses" ON "Addresses".email_addresses LIKE users.name || :name_1'

Here we can see that the where statement has been created with the specified expression we have inserted.

We can now include this with a select query to merge multiple tables.

In [103]:
s = select(users,
           addresses.c.email_addresses).select_from(users.join(addresses))
conn.execute(s).fetchall()

2021-06-01 21:35:21,282 INFO sqlalchemy.engine.Engine SELECT users.id, users.name, users.fullname, "Addresses".email_addresses 
FROM users JOIN "Addresses" ON users.id = "Addresses".user_id
2021-06-01 21:35:21,282 INFO sqlalchemy.engine.Engine [generated in 0.00079s] ()


[(1, 'Adam', 'Adam Borlase', 'scod101@gmail.com'),
 (1, 'Adam', 'Adam Borlase', 'a.c.borlase@gmail.com'),
 (2, 'Dominique', 'Dominique Freeborough', 'Name.Surname@gmail.com'),
 (1, 'Adam', 'Adam Borlase', 'scod101@gmail.com'),
 (1, 'Adam', 'Adam Borlase', 'a.c.borlase@gmail.com'),
 (2, 'Dominique', 'Dominique Freeborough', 'Name.Surname@gmail.com'),
 (1, 'Adam', 'Adam Borlase', 'scod101@gmail.com'),
 (1, 'Adam', 'Adam Borlase', 'a.c.borlase@gmail.com'),
 (2, 'Dominique', 'Dominique Freeborough', 'Name.Surname@gmail.com'),
 (1, 'Adam', 'Adam Borlase', 'scod101@gmail.com'),
 (1, 'Adam', 'Adam Borlase', 'a.c.borlase@gmail.com'),
 (2, 'Dominique', 'Dominique Freeborough', 'Name.Surname@gmail.com'),
 (1, 'Adam', 'Adam Borlase', 'scod101@gmail.com'),
 (1, 'Adam', 'Adam Borlase', 'a.c.borlase@gmail.com'),
 (2, 'Dominique', 'Dominique Freeborough', 'Name.Surname@gmail.com')]

The other join method that is avaliable to us is also `FromClause.outerjoin()` which creates a left outer join construct.
